In [36]:
import pandas as pd
import sqlite3 as sqlite


# Carga de datos

In [37]:
# Players DataFrame
players_df = pd.read_csv('players_stats.csv')
players_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column

# Teams Abreviation DataFrame
teams_abreviation_df = pd.read_csv('team_names.csv')
teams_abreviation_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column

# Teams DataFrame
teams_df = pd.read_csv('team_stats.csv')
teams_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column
teams_df.insert(1, "Key",teams_abreviation_df['Key'], False) 

# Rookie DataFrame
rookies_df = pd.read_csv('rookie_stats.csv')
rookies_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column


# Limpieza de datos

## Borrado de columnas que no vamos a usar

In [38]:
players_df = players_df.drop('eFG%',1)
teams_df = teams_df.drop(['Div', 'MOV/A', 'ORtg/A', 'DRtg/A', 'NRtg/A'],1)
rookies_df = rookies_df.drop(['Yrs','MP.1', 'PTS.1', 'TRB.1', 'AST.1'], 1)

## Borrado de filas con valores nulos

In [39]:
players_df = players_df.dropna()
teams_df = teams_df.dropna()
rookies_df = rookies_df.dropna()

## Eliminación de los jugadores que no tiene un equipo válido
El equipo de cada jugador debe estar en el dataframe de equipos identicado con el acrónimo correspondiente. Aunque, hay un caso excepcional, cuando un jugador ha estado en varios equipos durante la temporada, se guardan las estadistícas por separado y en total. En este último caso el equipo se identifica con TOT.

In [40]:
for i, row in players_df.iterrows():
    if row['Tm'] == 'TOT':
        continue
    
    if row['Tm'] not in list(teams_df['Key']):
        players_df = players_df.drop(index=i)

## Reiniciar índice 
El índice empieza en 1 y para evitar errores debe empezar en 0.

In [41]:
rookies_df = rookies_df.reset_index().drop('index',1)

## Creación del dataframe de la tabla central

In [48]:
central_df = pd.DataFrame(columns=['id_player', 'id_team', 'Player', 'Team','Rookie'])

central_df['id_player'] = players_df['id']
central_df['Player'] = players_df['Player']

for i, row in central_df.iterrows():
    # id team
    if players_df.loc[i]['Tm'] == 'TOT':
        row['id_team'] = -1 # id for total statistics
    else:
        row['id_team'] = teams_df.loc[list(teams_df['Key']).index(players_df.loc[i]['Tm'])]['id']


    # team name
    if row['id_team'] == -1:
        row['Team']='Total'
    else:
        row['Team'] = teams_df.loc[list(teams_df['id']).index(row['id_team'])]['Team']


    # is rookie
    if row['Player'] in list(rookies_df['Player']):
        row['Rookie'] = int(rookies_df.loc[list(rookies_df['Player']).index(row['Player'])]['id'])

    central_df.loc[i] = row

In [46]:
teams_df

,id,Key,Team,Conf,W,L,W/L%,MOV,ORtg,DRtg,NRtg
0,26,ATL,Atlanta Hawks,E,29.0,53.0,0.354,-6.02,108.71,114.41,-5.70
1,8,BOS,Boston Celtics,E,49.0,33.0,0.598,4.44,112.78,108.35,4.43
2,16,BKN,Brooklyn Nets,E,42.0,40.0,0.512,-0.07,110.30,110.36,-0.06
3,23,CHA,Charlotte Hornets,E,39.0,43.0,0.476,-1.10,112.10,113.16,-1.06
4,27,CHI,Chicago Bulls,E,22.0,60.0,0.268,-8.41,105.45,113.94,-8.49
5,30,CLE,Cleveland Cavaliers,E,19.0,63.0,0.232,-9.61,108.50,118.54,-10.04
6,19,DAL,Dallas Mavericks,W,33.0,49.0,0.402,-1.28,110.19,111.41,-1.22
7,7,DEN,Denver Nuggets,W,54.0,28.0,0.659,3.95,113.76,109.62,4.14
8,17,DET,Detroit Pistons,E,41.0,41.0,0.500,-0.24,109.93,110.13,-0.20
9,2,GSW,Golden State Warriors,W,57.0,25.0,0.695,6.46,116.63,110.24,6.39


# Creación de la base de datos

In [62]:
#Create BD connectiom
sql_data = 'nba_data.db'
connection = sqlite.connect(sql_data) 

In [63]:
#Create de execution cursor (Not needed for the moment)
cursor = connection.cursor() 
cursor.executescript('''
CREATE TABLE "PLAYERS" (
	"id"	INTEGER,
	"Player"	TEXT,
	"Pos"	TEXT,
	"Age"	REAL,
	"Tm"	TEXT,
	"G"	REAL,
	"GS"	REAL,
	"MP"	REAL,
	"FG"	REAL,
	"FGA"	REAL,
	"FG%"	REAL,
	"3P"	REAL,
	"3PA"	REAL,
	"3P%"	REAL,
	"2P"	REAL,
	"2PA"	REAL,
	"2P%"	REAL,
	"FT"	REAL,
	"FTA"	REAL,
	"FT%"	REAL,
	"ORB"	REAL,
	"DRB"	REAL,
	"TRB"	REAL,
	"AST"	REAL,
	"STL"	REAL,
	"BLK"	REAL,
	"TOV"	REAL,
	"PF"	REAL,
	"PTS"	REAL,
	PRIMARY KEY("id")
);

CREATE TABLE "ROOKIES" (
	"id"	INTEGER,
	"Player"	TEXT,
	"Debut"	TEXT,
	"Age"	REAL,
	"G"	REAL,
	"MP"	REAL,
	"FG"	REAL,
	"FGA"	REAL,
	"3P"	REAL,
	"3PA"	REAL,
	"FT"	REAL,
	"FTA"	REAL,
	"ORB"	REAL,
	"TRB"	REAL,
	"AST"	REAL,
	"STL"	REAL,
	"BLK"	REAL,
	"TOV"	REAL,
	"PF"	REAL,
	"PTS"	REAL,
	"FG%"	REAL,
	"3P%"	REAL,
	"FT%"	REAL,
	PRIMARY KEY("id")
);

CREATE TABLE "TEAMS" (
	"id"	INTEGER,
	"Key"	TEXT,
	"Team"	TEXT,
	"Conf"	TEXT,
	"W"	REAL,
	"L"	REAL,
	"W/L%"	REAL,
	"MOV"	REAL,
	"ORtg"	REAL,
	"DRtg"	REAL,
	"NRtg"	REAL,
	PRIMARY KEY("id")
);

CREATE TABLE "PLAYER_TEAM" (
	"id_player"	INTEGER,
	"id_team"	INTEGER,
	"Player"	TEXT,
	"Team"	TEXT,
	"Rookie"	INTEGER,
	FOREIGN KEY("id_team") REFERENCES "TEAMS"("id"),
	PRIMARY KEY("id_player","id_team"),
	FOREIGN KEY("id_player") REFERENCES "PLAYERS"("id"),
	FOREIGN KEY("Rookie") REFERENCES "ROOKIES"("id")
);

'''
)

In [64]:

#Load players_df in DB
players_df.to_sql('PLAYERS', connection, if_exists='append', index=False) 

#Load teams_df in DB
teams_df.to_sql('TEAMS', connection, if_exists='append', index=False)

#Load rookies_df in DB
rookies_df.to_sql('ROOKIES', connection, if_exists='append', index=False)

#Load central_df in DB
central_df.to_sql('PLAYER_TEAM', connection, if_exists='append', index=False)

#Show that the loading is been successfully
print(pd.read_sql("SELECT * FROM PLAYER_TEAM WHERE Rookie NOT NULL ;", connection))



#Close the connection with the DB
connection.close() 



id_player  id_team                  Player                  Team  Rookie
0           2       26            Jaylen Adams         Atlanta Hawks       1
1           5       30               Deng Adel   Cleveland Cavaliers       2
2           6        7  DeVaughn Akoon-Purcell        Denver Nuggets       3
3           8       27            Rawle Alkins         Chicago Bulls       4
4           9        4           Grayson Allen             Utah Jazz       5
..        ...      ...                     ...                   ...     ...
73        698       24           Yuta Watanabe     Memphis Grizzlies     108
74        699        7            Thomas Welsh        Denver Nuggets     111
75        707       22      Johnathan Williams    Los Angeles Lakers     112
76        708       21        Kenrich Williams  New Orleans Pelicans     113
77        725       26              Trae Young         Atlanta Hawks     115

[78 rows x 5 columns]
